In [1]:
import pandas as pd
import numpy as np

# Read in the data with the first two lines as a multi-index and the 'Player ID' column as the index
df = pd.read_csv('../../data/interim/nba_draft_data_combined_teams_cleaned.csv', header=[0,1], index_col=0)

# replace all 0s with NaNs
df = df.replace(0, np.nan)

df.head()

Career    Career                                                           \
Player ID Season Pk   Tm               Player       College   Yrs       G   
947.0       1996  1  PHI        Allen Iverson    Georgetown  14.0   914.0   
948.0       1996  2  TOR         Marcus Camby         UMass  17.0   973.0   
949.0       1996  3  VAN  Shareef Abdur-Rahim    California  12.0   830.0   
950.0       1996  4  MIN      Stephon Marbury  Georgia Tech  13.0   846.0   
951.0       1996  5  MIL            Ray Allen         UConn  18.0  1300.0   

Career                               ... 2021-22 2022-23                     \
Player ID       MP      PTS     TRB  ... TEAM_ID     AGE DEF_RATING  GP MIN   
947.0      37584.0  24368.0  3394.0  ...     NaN     NaN        NaN NaN NaN   
948.0      28684.0   9262.0  9513.0  ...     NaN     NaN        NaN NaN NaN   
949.0      28882.0  15028.0  6239.0  ...     NaN     NaN        NaN NaN NaN   
950.0      31891.0  16297.0  2516.0  ...     NaN     NaN        NaN NaN NaN   
951.0      46344.0  24505.0  5272.0  ...     NaN     NaN        NaN NaN NaN   

Career                                                         
Player ID NET_RATING OFF_RATING PIE TEAM_ABBREVIATION TEAM_ID  
947.0            NaN        NaN NaN                 0     NaN  
948.0            NaN        NaN NaN                 0     NaN  
949.0            NaN        NaN NaN                 0     NaN  
950.0            NaN        NaN NaN                 0     NaN  
951.0            NaN        NaN NaN                 0     NaN  

[5 rows x 265 columns]

In [2]:
# Get the position of the last career column
position_to_insert = len(df['Career'].columns)
# Get the list of columns to insert
columns_to_insert = [('Career', 'OFF_RATING'), ('Career', 'DEF_RATING'), ('Career', 'NET_RATING'), ('Career', 'PIE')]

# Insert multiple columns with NaN values
for col in columns_to_insert:
    df.insert(loc=position_to_insert, column=col, value=np.nan)

In [3]:
# create list with all seasons between 1996-97 and 2022-23
seasons = []
for i in range(1996, 2023):
    seasons.append(str(i) + '-' + str(i+1)[2:])

# for each season, insert a new column with the product of 'GP' and 'OFF_RATING'
for season in seasons:
    # find the (season, 'OFF_RATING') column as the position to insert the new column
    position_to_insert = df.columns.get_loc((season, 'OFF_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'OFF_RATING_weighted'), value=np.nan)
    # fill the new column with the product of (season,'GP') and (season,'OFF_RATING')
    df[(season, 'OFF_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'OFF_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'DEF_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'DEF_RATING_weighted'), value=np.nan)
    df[(season, 'DEF_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'DEF_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'NET_RATING'))
    df.insert(loc=position_to_insert, column=(season, 'NET_RATING_weighted'), value=np.nan)
    df[(season, 'NET_RATING_weighted')] = df[(season, 'GP')] * df[(season, 'NET_RATING')]
    
    position_to_insert = df.columns.get_loc((season, 'PIE'))
    df.insert(loc=position_to_insert, column=(season, 'PIE_weighted'), value=np.nan)
    df[(season, 'PIE_weighted')] = df[(season, 'GP')] * df[(season, 'PIE')]

/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_31052/1738369481.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=position_to_insert, column=(season, 'DEF_RATING_weighted'), value=np.nan)
/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_31052/1738369481.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=position_to_insert, column=(season, 'NET_RATING_weighted'), value=np.nan)
/var/folders/zb/fj1ym99j70b38snb8v3_0g580000gn/T/ipykernel_31052/1738369481.py:23: PerformanceWarnin

In [4]:
# in the ('Career', rating) column, fill the NaNs with the sum of the weighted columns divided by the ('Career', 'G') column
df[('Career', 'OFF_RATING')] = df.filter(regex='OFF_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'DEF_RATING')] = df.filter(regex='DEF_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'NET_RATING')] = df.filter(regex='NET_RATING_weighted').sum(axis=1) / df[('Career', 'G')]
df[('Career', 'PIE')] = df.filter(regex='PIE_weighted').sum(axis=1) / df[('Career', 'G')]
# drop the weighted columns
df = df.drop(df.filter(regex='_weighted').columns, axis=1)

In [5]:
# copy the 'Career' columns to a new dataframe
df_career = df['Career'].copy()

# export the dataframe to a csv file
df_career.to_csv('../../data/processed/player_career_avg.csv')